# Basic test of the openai library

The first snippet executes a sample prompt to openai directly using their library.

Here we tell the model to act as if it were a poetic assistant with programming knowledge. Then we ask it to write a poem about recursion. The model then generates a poem about recursion in programming.

In [4]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message.content)

In the realm of code, a curious thing,
A pattern where function within does spring.
Recursion, it's called, a loop unbound,
The art of repeating without hitting the ground.

Invoke it once, and then again,
A function calls itself, an endless chain.
Through problems complex, it weaves and winds,
Solutions in its labyrinthine binds.

A base case first, so crucial, so needful,
To halt the cycle, lest it grows too heedful.
For without this check, this vital break,
An infinite loop our code will make.

Like Russian dolls, one within the next,
Functions unfold, their contents complex.
Each layer peels to reveal one more,
Till the smallest one shows the core.

A fibonacci sequence, elegantly unfurled,
Or traversing trees, roots to leaves unfurled.
Recursion handles such tasks with grace,
Processing each step, each recursive trace.

Think of it as a spell, cast and recast,
In an enchanter's chant, the magic is vast.
But beware the charm, its seductive song,
For without care, things may go wron

# Intermediate test of the openai library, now with langchain

This second snippet is a bit more sophisticated and uses langchain to execute a prompt.

Here we tell the model to write a summary given a specific topic (again, recursion). The model then generates a summary about the topic.

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

summary_template = """
    given a topic {topic}, write a summary of the topic in your own words.
"""

summary_prompt = PromptTemplate.from_template(template=summary_template)

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

chain = LLMChain(llm=llm, prompt=summary_prompt)

print(chain.run(topic="recursion in programming"))


Recursion in programming is a method where a function calls itself directly or indirectly to solve a problem. It's a powerful tool that simplifies the code for tasks that can be broken down into similar subtasks. Recursion is often used in algorithms that deal with data structures like trees and graphs, as well as in sorting algorithms like quicksort and mergesort.

A recursive function typically has two main components: the base case and the recursive case. The base case defines the condition under which the recursion will stop, preventing an infinite loop of function calls. The recursive case is where the function calls itself with a modified argument, gradually approaching the base case.

For example, a classic instance of recursion is the calculation of factorial, where the factorial of a number n (denoted as n!) is the product of all positive integers less than or equal to n. The base case is when n equals 0 or 1, in which case the factorial is 1. The recursive case involves multi

# Complext test of the openai library, now with langchain and map-reduce

The final snippet is the most sophisticated one.

It first retrieves a list of news articles from google cloud blog RSS feed and load each article as a Human Readable text using nltk. Then it uses langchain to execute a map-reduce  algorithm which maps each article into a summary and then reduces them all to generate an output with the highlights of the google cloud blog latest news.

In [3]:
import nltk

from functools import partial
from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import RSSFeedLoader
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser, Document
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

nltk.download('punkt')

urls = ["https://cloudblog.withgoogle.com/rss"]

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

loader = RSSFeedLoader(urls=urls, nlp=True)
data = loader.load()

document_prompt = PromptTemplate.from_template("{page_content}")
partial_format_document = partial(format_document, prompt=document_prompt)

map_chain = (
    {"context": partial_format_document}
    | PromptTemplate.from_template("Summarize this news article:\n\n{context}")
    | llm
    | StrOutputParser()
)

map_as_doc_chain = (
    RunnableParallel({"doc": RunnablePassthrough(), "content": map_chain})
    | (lambda x: Document(page_content=x["content"], metadata=x["doc"].metadata))
).with_config(run_name="Summarize (return doc)")

def format_docs(docs):
    return "\n\n".join(partial_format_document(doc) for doc in docs)

collapse_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Collapse this content:\n\n{context}")
    | llm
    | StrOutputParser()
)

def get_num_tokens(docs):
    return llm.get_num_tokens(format_docs(docs))

def collapse(
    docs,
    config,
    token_max=4000,
):
    collapse_ct = 1
    while get_num_tokens(docs) > token_max:
        config["run_name"] = f"Collapse {collapse_ct}"
        invoke = partial(collapse_chain.invoke, config=config)
        split_docs = split_list_of_docs(docs, get_num_tokens, token_max)
        docs = [collapse_docs(_docs, invoke) for _docs in split_docs]
        collapse_ct += 1
    return docs

reduce_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Based on these news summaries:\n\n{context}\n\n Tell me latest news from Google Cloud.")
    | llm
    | StrOutputParser()
).with_config(run_name="Reduce")

map_reduce = (map_as_doc_chain.map() | collapse | reduce_chain).with_config(
    run_name="Map reduce"
)

print(map_reduce.invoke(data[0:5], config={"max_concurrency": 5}))


[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


As of my last update, here are some of the latest developments from Google Cloud:

1. **Enhancements to Cybersecurity with ISACs**: Google Cloud has been actively supporting Information Sharing and Analysis Centers (ISACs) to improve cybersecurity across various sectors. By sharing threat intelligence, including strategies and methods used by attackers, organizations can respond more effectively to cyber threats. Google Cloud's partnership with Health-ISAC, for example, has led to an open-source integration that connects the Health-ISAC Indicator Threat Sharing (HITS) feed with Google's Chronicle Security Operations platform, allowing for rapid sharing and detection of cyber threats.

2. **Investment in Cybersecurity**: Google has pledged to invest $10 billion over five years in cybersecurity advancements. This investment includes supporting ISACs and providing threat intelligence from Google services like Mandiant and VirusTotal to ISAC members at no extra cost.

3. **Custom Queues fo